In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time


## VS Code, choisir un seul des 3 :

# Pour affichage interactif (notamment 3D) dans notebook
# %matplotlib widget

# Pour affichage interactif (notamment 3D) dans une fenêtre qt externe au notebook
# %matplotlib qt

# Pour affichage simple dans notebook
%config InlineBackend.figure_format = 'png'

t = time.time()

In [2]:
df = pd.read_csv('data/df_train.csv')
df.columns

Index(['Unnamed: 0', 'day', 'hashed_partner_id', 'hashed_campaign_id',
       'contextid', 'display_env', 'target_env', 'rtbtypeid',
       'rtbadvisibility', 'rtb_detectedlanguage', 'urlhash2', 'urlhash3',
       'urlhash4', 'user_country', 'hashed_affiliateid', 'hashed_app_id',
       'googleviewability', 'googlepagevertical', 'campaignscenario',
       'campaignvertical', 'campaignctrlast24h', 'is_interstitial',
       'dayssincelastvisitdouble', 'ltf_lastpartnerclicktimestamp',
       'ltf_nbglobalclick_4w', 'ltf_nbglobaldisplay_4w',
       'ltf_nbglobaldisplaysincelastpartnerproductview',
       'ltf_nbpartnerdisplayssincelastclick', 'ltf_nbpartnerclick_4w',
       'ltf_nbpartnerdisplay_4w', 'ltf_nbpartnersales_4w',
       'ltf_nbpartnerdisplay_90d', 'ltf_nbpartnerclick_90d',
       'ltf_nbpartnersales_90d', 'nbdayssincelastclick', 'nbdisplay_1hour',
       'nbdisplaypartnerapprox_1d_sum_xdevice',
       'nbdisplayaffiliateapprox_1d_sum_xdevice',
       'nbdisplayglobalapprox_1d_s

In [3]:
# Recodage de certaines modalités afin de réordonner par ordre croissant d'engagement
df["contextid"] = df["contextid"].replace(0,1)
df["contextid"] = df["contextid"].replace(10,3)

# Remplacement des valeurs manquantes par des 0 dans les variables commençant par ltf
df['ltf_lastpartnerclicktimestamp'] = df['ltf_lastpartnerclicktimestamp'].fillna(0)
df['ltf_nbglobalclick_4w'] = df['ltf_nbglobalclick_4w'].fillna(0)
df['ltf_nbglobaldisplay_4w'] = df['ltf_nbglobaldisplay_4w'].fillna(0)
df['ltf_nbglobaldisplaysincelastpartnerproductview'] = df['ltf_nbglobaldisplaysincelastpartnerproductview'].fillna(0)
df['ltf_nbpartnerdisplayssincelastclick'] = df['ltf_nbpartnerdisplayssincelastclick'].fillna(0)
df['ltf_nbpartnerclick_4w'] = df['ltf_nbpartnerclick_4w'].fillna(0)
df['ltf_nbpartnerdisplay_4w'] = df['ltf_nbpartnerdisplay_4w'].fillna(0)
df['ltf_nbpartnersales_4w'] = df['ltf_nbpartnersales_4w'].fillna(0)
df['ltf_nbpartnerdisplay_90d'] = df['ltf_nbpartnerdisplay_90d'].fillna(0)
df['ltf_nbpartnerclick_90d'] = df['ltf_nbpartnerclick_90d'].fillna(0)
df['ltf_nbpartnersales_90d'] = df['ltf_nbpartnersales_90d'].fillna(0)
# Idem ici
df['nbdisplay_1hour'] = df['nbdisplay_1hour'].fillna(0)
df['nbdisplaypartnerapprox_1d_sum_xdevice'] = df['nbdisplaypartnerapprox_1d_sum_xdevice'].fillna(0)
df['nbdisplayaffiliateapprox_1d_sum_xdevice'] = df['nbdisplayaffiliateapprox_1d_sum_xdevice'].fillna(0)
df['nbdisplayglobalapprox_1d_sum_xdevice'] = df['nbdisplayglobalapprox_1d_sum_xdevice'].fillna(0)	
df['campaignctrlast24h'] = df['campaignctrlast24h'].fillna(0)

In [4]:
# Variables avec valeurs manquantes
pd.DataFrame({"Valeurs manquantes":df[df.columns[df.isnull().sum() != 0]].isnull().sum(), "Proportion manquante":df[df.columns[df.isnull().sum() != 0]].isnull().sum()/len(df)})

,Valeurs manquantes,Proportion manquante
rtbtypeid,9066,0.004887
rtbadvisibility,9066,0.004887
rtb_detectedlanguage,9066,0.004887
urlhash2,155695,0.083918
urlhash3,155695,0.083918
urlhash4,155695,0.083918
user_country,366,0.000197
hashed_app_id,736418,0.396924
googleviewability,1278860,0.689296
googlepagevertical,9066,0.004887


In [5]:
# Suppression de l'appareil cliquant un nombre aberrant de fois
df.drop(df.index[df['hashed_xd_id'] == r"b'\x12\xb97|\xbe~\\\x94\xe8\xa7\r\x9d#\x92\x95#\xd1J\xfa\x95G\x93\x13\x0f\x8a9Y\xc7\xb8I\xac\xa8'"], inplace = True)

In [6]:
# Les nombres de clics ne sont plus aberrants
b = df[['is_display_clicked', 'hashed_xd_id']].groupby(['hashed_xd_id']).sum()
b.sort_values('is_display_clicked', ascending=False)

,is_display_clicked
hashed_xd_id,
b'\x97\x9d)\xb3*\x8e\xf6\xeer\xf7\x166a\x12\xcc\xa5\xf4\x90\xc9\xd9\x197\xd1\xdf\x17f\x10%\xfe\x99\xe4\xb1',10
"b""L\xf5H\x97\x10\xfd\x7f~\x1a9\xa3?\xac\xfdNf\xf6\\\x8f13\xc1\xcc\xf8\xbf\xc9q\x87'\xbc]H""",10
b'\xc4\xc8\xb7\xb0\x7faU\x01D5\xfe\x96\xb7\x165\xff\x90\xc2\xcd;\xbf^\\\x10\xf9z*\x0e\xc6%4v',9
b'\xadk\x10\x99k\\\x7f\xef\xf3\x86\xe6f\\{)\xd8\x14t\x8b\x8b\x16\x7f\xb0z)\xd4\n\xe7&dN\xe9',9
b'>\x90\xdd\x05s\xf2\x8a\x1dv\xc8Kdy\xbc\xd9\xe7a+\xf6\x04G2\xed\x1eT\xe6\xaeS\x81L$\x9c',9
...,...
b'\x05\xa3\x85\xf7TeZ\x98\xcc&\x15\xdc\xbaY\x85\xe8\xf3\xce\xd2I\xc8R:;-\x1f\xb9\xb9\x9e\xef\x89\x95',0
"b'\x05\xa3\x99\xafx4\x96,G\xca\x03I\x97C|\xe7~\xad\x1d\x80M\x96[\xf0\xbd\x886\x96\x1c\xb8\x9c\xa9'",0
b'\x05\xa3\xaa?\xa0W\xf5\xc2\xf4\xcev~X\xe6\xfc\xb3%\xc8?;T\xff\xc1\x02\xf5\\k\xcb2N[\n',0


In [7]:
df['display_size'] = df['display_width']*df['display_height']

In [8]:
df.to_csv('data/df_train_prepro.csv')

In [9]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")

Temps d'exécution total : 00:01:35
